In [1]:
from pathlib import Path

spark.catalog.setCurrentCatalog('workspace')
spark.catalog.setCurrentDatabase('default')


Databricks Connect detected
Databricks Connect detected
+-----------------------+
|'dbconnect established'|
+-----------------------+
|dbconnect established  |
+-----------------------+



In [2]:
# CONFIGS: please adjust to your volume
PATH_DATASETS = Path('/Volumes/workspace/default/data')

In [3]:
# recipe data
df_recipe = spark.read.csv((path_demodata / 'recipeData.csv').as_posix(), header=True, inferSchema=True)
df_recipe = df_recipe.dropna().withColumnRenamed('Size(L)', 'Size')

df_recipe.write.saveAsTable('fct_recipe', mode="overwrite", overwriteSchema=True)
df_style = df_recipe.select("StyleID", "Style").distinct()
df_style.write.saveAsTable('dim_style', mode="overwrite", overwriteSchema=True)



In [8]:
# fraud data
csv_reader = spark.read.format('csv').options(header='true', inferSchema='true')
df_fraud = csv_reader.load((path_demodata / 'bs140513_032310.csv').as_posix())
df_fraud.write.saveAsTable('bs140513_032310', mode='overwrite')


SparkConnectException: [NO_ACTIVE_SESSION] No active Spark session found. Please create a new Spark session before running the code.

In [4]:
# book data
csv_reader = spark.read.format('csv').options(header='true', inferSchema='true')
df_books = csv_reader.options(**{"escape": "\"", "quote": "\""}).load((path_demodata / 'books.csv').as_posix())
df_book_tags = csv_reader.load((path_demodata / 'book_tags.csv').as_posix())
df_ratings = csv_reader.load((path_demodata / 'ratings.csv').as_posix())
df_tags = csv_reader.load((path_demodata / 'tags.csv').as_posix())
df_toread = csv_reader.load((path_demodata / 'to_read.csv').as_posix())

df_books.write.saveAsTable('books', overwriteSchema=True, mode="overwrite")
df_book_tags.write.saveAsTable('book_tags', overwriteSchema=True, mode="overwrite")
df_ratings.write.saveAsTable('ratings', overwriteSchema=True, mode="overwrite")
df_tags.write.saveAsTable('tags', overwriteSchema=True, mode="overwrite")
df_toread.write.saveAsTable('to_read', overwriteSchema=True, mode="overwrite")


In [5]:
# optional add keys
spark.sql("ALTER TABLE fct_recipe ALTER COLUMN BeerID SET NOT NULL;")
spark.sql("ALTER TABLE fct_recipe ADD CONSTRAINT beer_pk PRIMARY KEY (BeerID) RELY")
spark.sql("ALTER TABLE dim_style ALTER COLUMN StyleID SET NOT NULL;")
spark.sql("ALTER TABLE dim_style ADD CONSTRAINT style_pk PRIMARY KEY (StyleID) RELY")
spark.sql("ALTER TABLE fct_recipe ADD CONSTRAINT style_fk FOREIGN KEY (StyleID) REFERENCES dim_style(StyleID)")


DataFrame[]

In [6]:
spark.sql("ALTER TABLE books ALTER COLUMN book_id SET NOT NULL;")
spark.sql("ALTER TABLE books ADD CONSTRAINT book_id_pk PRIMARY KEY (book_id) RELY")
spark.sql("ALTER TABLE tags ALTER COLUMN tag_id SET NOT NULL;")
spark.sql("ALTER TABLE tags ADD CONSTRAINT tag_id_pk PRIMARY KEY (tag_id) RELY")
spark.sql("ALTER TABLE book_tags ALTER COLUMN goodreads_book_id SET NOT NULL;")
spark.sql("ALTER TABLE book_tags ALTER COLUMN tag_id SET NOT NULL;")
spark.sql("ALTER TABLE book_tags ADD CONSTRAINT booktag_id_pk PRIMARY KEY (goodreads_book_id, tag_id) RELY")
spark.sql("ALTER TABLE book_tags ADD CONSTRAINT booktag_tagid_fk FOREIGN KEY (tag_id) REFERENCES tags(tag_id)")
spark.sql("ALTER TABLE book_tags ADD CONSTRAINT booktag_bookid_fk FOREIGN KEY (goodreads_book_id) REFERENCES books(book_id)")
spark.sql("ALTER TABLE ratings ALTER COLUMN book_id SET NOT NULL;")
spark.sql("ALTER TABLE ratings ALTER COLUMN user_id SET NOT NULL;")
spark.sql("ALTER TABLE ratings ADD CONSTRAINT ratingsuser_id_pk PRIMARY KEY (book_id, user_id) RELY")
spark.sql("ALTER TABLE ratings ADD CONSTRAINT ratingsuser_id_fk  FOREIGN KEY (book_id) REFERENCES books(book_id) ")



DataFrame[]